In [2]:
from google.colab import files

# Upload a file from your local machine to Colab
uploaded = files.upload()

Saving 0-Annotated.json to 0-Annotated.json
Saving 1-Annotated.json to 1-Annotated.json
Saving 2-Annotated.json to 2-Annotated.json
Saving 3-Annotated.json to 3-Annotated.json
Saving 4-Annotated.json to 4-Annotated.json
Saving 5-Annotated.json to 5-Annotated.json
Saving 6-Annotated.json to 6-Annotated.json
Saving 7-Annotated.json to 7-Annotated.json
Saving 8-Annotated.json to 8-Annotated.json
Saving 9-Annotated.json to 9-Annotated.json
Saving 10-Annotated.json to 10-Annotated.json


In [72]:
file_paths = [f"{i}-Annotated.json" for i in range(0, 11)]

import json

file_content_dict = {}
for i, file_path in enumerate(file_paths):
  with open(file_path, 'r') as file:
      file_content = file.read()
      file_content_dict[i] = json.loads(file_content)
# papers for hand gesture capture/data acquisition
# existing hand gestures datasets
# leap motion controller based datasets
# understand leap motion sensor working

In [73]:
import pandas as pd

data_list = []
for i in range(0, 11):
  for j in range(0, len(file_content_dict[i])):
    data_list.append(file_content_dict[i][j])

features_list = []
labels_list = []
for data_point in data_list:
    features = []
    features.extend(data_point["FrameData"][0]["HandPosition"])
    features.extend(data_point["FrameData"][0]["HandRotation"])
    finger_positions = data_point["FrameData"][0]["FingerPositions"]
    for position in finger_positions:
        features.extend(position)

    hand_id = data_point["FrameData"][0]["HandID"]
    features.append(hand_id)

    orientation = data_point["FrameData"][0]["HandOrientation"]
    features.append(orientation)

    label = data_point["Gesture"]

    features_list.append(features)
    labels_list.append(label)

feature_names = ["HandPositionX", "HandPositionY", "HandPositionZ",
                 "HandRotation1", "HandRotation2", "HandRotation3", "HandRotation4"]
for i in range(1, 6):
    feature_names.extend([f"FingerPosition{i}X", f"FingerPosition{i}Y", f"FingerPosition{i}Z"])

feature_names.append("HandID")
feature_names.append("HandOrientation")
df = pd.DataFrame(features_list, columns=feature_names)
df["Gesture"] = labels_list

df = pd.get_dummies(df, columns=["HandOrientation", "HandID"])
df = df.sample(frac = 1)


In [80]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder, StandardScaler

X = df.drop(columns=["Gesture"])
y = df["Gesture"]

X, y = shuffle(X, y)

scaler = StandardScaler()
X = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

le = LabelEncoder()
y = pd.Series(le.fit_transform(y))

X_train = pd.DataFrame(columns=X.columns)
X_test = pd.DataFrame(columns=X.columns)
y_train = pd.Series(dtype="str")
y_test = pd.Series(dtype="str")

test_size = 0.2

unique_labels = y.unique()
for label in unique_labels:
    label_data_X = X[y.reindex(X.index) == label]
    label_data_y = y[y.reindex(X.index) == label]

    X_label_train, X_label_test, y_label_train, y_label_test = train_test_split(
        label_data_X, label_data_y, test_size=test_size)

    X_train = pd.concat([X_train, X_label_train])
    X_test = pd.concat([X_test, X_label_test])
    y_train = pd.concat([y_train, y_label_train])
    y_test = pd.concat([y_test, y_label_test])


In [81]:
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import confusion_matrix

clf = SVC()

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)
# cm = confusion_matrix(y_test, y_pred)
# print(cm)
print("Training Accuracy:",metrics.accuracy_score(y_train, y_train_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Training Accuracy: 0.9953271028037384
Accuracy: 0.9814814814814815


In [82]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier()

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)

print("Training Accuracy:",metrics.accuracy_score(y_train, y_train_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Training Accuracy: 1.0
Accuracy: 0.9444444444444444


In [83]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()

clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)

print("Training Accuracy:",metrics.accuracy_score(y_train, y_train_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Training Accuracy: 1.0
Accuracy: 1.0


In [84]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier()

clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)

print("Training Accuracy:",metrics.accuracy_score(y_train, y_train_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Training Accuracy: 0.985981308411215
Accuracy: 0.9444444444444444


In [85]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier()

clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

y_train_pred = clf.predict(X_train)

print("Training Accuracy:",metrics.accuracy_score(y_train, y_train_pred))
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Training Accuracy: 1.0
Accuracy: 0.9629629629629629
